# Bike-Share — Large CSV Exploration + Missing Values

This notebook uses a synthetic dataset (Berlin, Munich, Hamburg) with ~120k trips.
It demonstrates chunked CSV reading, data abstraction summaries, missing-values workflow, and simple plots.

In [ ]:
# %pip install pandas matplotlib
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
pd.options.display.max_columns = 50
USE_LOCAL_FILE = True
LOCAL_PATH = Path('bikeshare_2024.csv')
CHUNKSIZE = 25_000
COLUMNS = ['trip_id','city','started_at','ended_at','start_station_id','end_station_id','start_lat','start_lng','end_lat','end_lng','user_type','bike_type','duration_s','distance_km','fare_eur']

## 1. Chunked schema-ish summary

In [ ]:
from collections import Counter
def chunk_reader():
    return pd.read_csv(LOCAL_PATH, usecols=COLUMNS, chunksize=CHUNKSIZE, low_memory=False)

n_rows=0; missing_counts=Counter()
min_start=None; max_start=None
lat_min=np.inf; lat_max=-np.inf; lng_min=np.inf; lng_max=-np.inf

for chunk in chunk_reader():
    n_rows += len(chunk)
    for col in ['started_at','ended_at']:
        chunk[col] = pd.to_datetime(chunk[col], errors='coerce')
    missing_counts.update(chunk.isna().sum().to_dict())
    if chunk['started_at'].notna().any():
        cmin, cmax = chunk['started_at'].min(), chunk['started_at'].max()
        min_start = cmin if (min_start is None or cmin < min_start) else min_start
        max_start = cmax if (max_start is None or cmax > max_start) else max_start
    if chunk['start_lat'].notna().any():
        lat_min = min(lat_min, float(chunk['start_lat'].min()))
        lat_max = max(lat_max, float(chunk['start_lat'].max()))
    if chunk['start_lng'].notna().any():
        lng_min = min(lng_min, float(chunk['start_lng'].min()))
        lng_max = max(lng_max, float(chunk['start_lng'].max()))

missing_pct = {k: (v / n_rows) * 100 for k, v in missing_counts.items()}
print('Rows processed:', n_rows)
print('Temporal coverage:', min_start, '→', max_start)
print('Spatial extent (approx): lat', lat_min, 'to', lat_max, '; lng', lng_min, 'to', lng_max)
print('Missing values (%):')
missing_pct

## 2. Build a working sample (~50k rows)

In [ ]:
SAMPLE_ROWS = 50_000
sample=[]
for chunk in pd.read_csv(LOCAL_PATH, usecols=COLUMNS, chunksize=CHUNKSIZE, low_memory=False):
    for col in ['started_at','ended_at']:
        chunk[col] = pd.to_datetime(chunk[col], errors='coerce')
    sample.append(chunk)
    if sum(len(s) for s in sample) >= SAMPLE_ROWS:
        break
df = pd.concat(sample, ignore_index=True).head(SAMPLE_ROWS)
df.shape, df.head(3)

## Helpful pandas function


In [ ]:
df['bike_type'].describe()

In [ ]:
df['duration_s'].describe()

## 3. Missing-values workflow

In [ ]:
crit_mask = df['started_at'].notna()
dfc = df.loc[crit_mask].copy()

for c in dfc.columns:
    dfc[c+'_was_missing'] = dfc[c].isna()
dfc['hour_of_day'] = dfc['started_at'].dt.hour
dfc['is_same_station'] = (dfc['start_station_id'] == dfc['end_station_id'])
cat_cols = ['city','user_type','bike_type','start_station_id','end_station_id']

for c in cat_cols:
    if c in dfc:
        mode = dfc[c].mode(dropna=True)
        dfc[c] = dfc[c].fillna(mode.iloc[0] if not mode.empty else 'Unknown')

for coord in ['start_lat','start_lng','end_lat','end_lng']:
    if coord in dfc:
        if 'city' in dfc:
            dfc[coord] = dfc.groupby('city')[coord].transform(lambda s: s.fillna(s.median()))
        dfc[coord] = dfc[coord].fillna(dfc[coord].median())

dfc.loc[dfc['fare_eur'] < 0, 'fare_eur'] = np.nan
dfc.loc[(dfc['duration_s'] <= 0) | (dfc['duration_s'] > 6*3600), 'duration_s'] = np.nan

for c in ['duration_s','distance_km','fare_eur']:
    if c in dfc:
        dfc[c] = dfc[c].fillna(dfc[c].median())
        
dfc.isna().mean().sort_values(ascending=False).head()

## 4. Quick plots

In [ ]:
plt.figure(); dfc['distance_km'].clip(upper=dfc['distance_km'].quantile(0.99)).hist(bins=40)
plt.title('Trip distance (km) — clipped 99th pct'); plt.xlabel('km'); plt.ylabel('count'); plt.show()
plt.figure(); dfc['hour_of_day'].value_counts().sort_index().plot(kind='bar')
plt.title('Trips by hour of day (sample)'); plt.xlabel('hour'); plt.ylabel('trips'); plt.show()
plt.figure(); dfc.groupby('city')['fare_eur'].mean().sort_values().plot(kind='bar')
plt.title('Average fare by city (sample)'); plt.xlabel('city'); plt.ylabel('fare_eur (avg)'); plt.show()

## 5. Export summaries & cleaned sample

In [ ]:
def summarize_col(s: pd.Series):
    out = {'dtype': str(s.dtype), 'missing_pct': float(s.isna().mean()*100)}
    if pd.api.types.is_datetime64_any_dtype(s):
        out.update({'type':'temporal','min': s.min(), 'max': s.max()})
    elif pd.api.types.is_numeric_dtype(s):
        out.update({'type':'quantitative','min': float(np.nanmin(s)), 'max': float(np.nanmax(s))})
    else:
        out.update({'type':'categorical','unique': int(s.nunique(dropna=True))})
    return out
schema_rows = []
for c in ['trip_id','city','started_at','ended_at','start_station_id','end_station_id','start_lat','start_lng','end_lat','end_lng','user_type','bike_type','duration_s','distance_km','fare_eur']:
    if c in dfc:
        schema_rows.append({'attribute': c, **summarize_col(dfc[c])})
schema_df = pd.DataFrame(schema_rows)
schema_df.to_csv('summary_bikeshare_sample.csv', index=False)
dfc.to_csv('bikeshare_cleaned_sample.csv', index=False)

schema_df.head(10)

In [ ]:
# Prepare series (coerce to string; keep a Missing bin)
s = df.get("bike_type", pd.Series(dtype=object)).astype("object").fillna("Missing")

as_percent = False  # set True to plot percentages
top_n = None        # set an int to cap categories shown, e.g., 10

vc = s.value_counts(dropna=False)
plot_data = (vc / len(s) * 100) if as_percent else vc
if top_n:
    plot_data = plot_data.head(top_n)

# Quick text summary
print(f"bike_type — unique(non-missing)={df['bike_type'].nunique(dropna=True) if 'bike_type' in df else 0}, "
      f"missing={(df['bike_type'].isna().mean()*100 if 'bike_type' in df else 0):.1f}%")
print(plot_data)

# Plot
plt.figure()
plot_data.plot(kind="bar")
plt.title(f"bike_type — {'%' if as_percent else 'count'} (n={len(s)})")
plt.xlabel("bike_type")
plt.ylabel("% of rows" if as_percent else "count")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()